In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

import torch
from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.datasets import register_coco_instances
from detectron2.data import get_detection_dataset_dicts
from detectron2.data.detection_utils import read_image
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode

import numpy as np
import os
import json
import cv2
from pycocotools import mask

import json
from detectron2.data.detection_utils import read_image
from PIL import Image
import numpy

import cv2
from matplotlib import pyplot as plt
from detectron2.utils.visualizer import Visualizer, VisImage

from detectron2.config import configurable
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads

from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, fast_rcnn_inference
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
from detectron2.layers import ShapeSpec, batched_nms
from typing import Dict, List, Optional, Tuple
from detectron2.modeling.poolers import ROIPooler
from torch.nn import functional as F

from detectron2.layers import ShapeSpec, batched_nms, cat, cross_entropy, nonzero_tuple
from detectron2.utils.events import get_event_storage
from detectron2.evaluation.coco_evaluation import COCOEvaluator

from detectron2.engine.defaults import DefaultTrainer
import logging
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.layers import ShapeSpec
import torch

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class NonCocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name)

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128


In [6]:
#training_categ = ['chess', 'skiing', 'weightlifting', 'climbing', 'cricket', 'flying', 'hockey', 'soccer', 'volleyball', 'tennis', 'skateboarding', 'swimming', 'rowing', 'roller skating', 'horse racing', 'steeplechase', 'jogging', 'gymnastics', 'golf', 'diving', 'car racing', 'boxing', 'bowling', 'billiard', 'beach volleyball', 'basketball', 'baseball', 'jumping', 'running', 'acrobatics', 'airplane', 'glider', 'helicopter', 'hot-air_balloon', 'bicycle', 'camper', 'convertible', 'jeep', 'limousine', 'sedan', 'taxi', 'wagon', 'carriage', 'motorcycle', 'bus', 'minibus', 'tram', 'trolleybus', 'road sign', 'traffic police', 'zebra crossing', 'boat', 'ferry', 'gondola', 'motorboat', 'sailing vessel', 'ship', 'yacht', 'sleigh', 'rocket', 'spaceship', 'train', 'car transporter', 'dumper', 'garbage truck', 'lorry', 'pickup', 'tow truck', 'truck', 'van', 'bulldozer', 'digger', 'forklift', 'tractor', 'artist', 'sculptor', 'accordionist', 'piper', 'cellist', 'clarinetist', 'conductor', 'flute player', 'guitar player', 'opera singer', 'percussionist', 'piano player', 'rapper', 'saxophonist', 'singer', 'trombonist', 'trumpeter', 'violinist', 'ballet dancer', 'cameraman', 'clown', 'dancer', 'makeup artist', 'photographer', 'writer', 'figure skating', 'off road motorcycling', 'motorcycle racing', 'baby carriage', 'fire engine', 'fireman', 'police car', 'police helicopter', 'mounted police', 'policeman', 'wheelchair', 'fishing', 'hunting', 'tank', 'hang gliding', 'rhythmic gymnastics', 'horse sleigh', 'ambulance', 'dog sleigh', 'military helicopter', 'police boat', 'motorcycle police', 'soldier', 'double-decker', 'bicycle racing', 'handball', 'armoured personnel carrier', 'military truck', 'rickshaw', 'scooter', 'pole vaulting']
training_categ = ['chess','car transporter','violinist','motorcycle racing','tank','baseball','helicopter','police helicopter','military helicopter','bicycle racing','cellist','fishing','hunting','handball','makeup artist','military truck','spaceship','pickup','skiing','bicycle racing']
training_categ = ['hunting','handball','makeup artist','military truck','spaceship','pickup','skiing','bicycle racing']
training_categ = ['figure skating']

cfg = get_cfg()
cfg.merge_from_file('/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml')
cfg.MODEL.WEIGHTS = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x/137260431/model_final_a54504.pkl'

cfg.DATALOADER.NUM_WORKERS = 1
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1400)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.TEST.EVAL_PERIOD = 2000
cfg.MODEL.DEVICE='cuda:1'

for folder_name in training_categ:
    folder_name = folder_name.replace(' ','_')
    try:
        DatasetCatalog.remove(folder_name)
        MetadataCatalog.remove(folder_name)
        print('Update dataset')
    except:
        print('New dataset')
    #register_coco_instances(folder_name,{},'/host/comparison/'+folder_name+'/'+folder_name+'_train.json','/host/comparison/'+folder_name+'/data/')
    register_coco_instances(folder_name,{},'/host/mic21-framework/server/uploads/'+folder_name+'_gt.json','/host/mic21-framework/server/uploads/'+folder_name+'/')
    
    try:
        del trainer
    except:
        print("Starting...")
    
    ind = 0
    img = DatasetCatalog.get(folder_name)[ind]['file_name']
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #v = Visualizer(img,MetadataCatalog.get(folder_name))
    #v.draw_dataset_dict(DatasetCatalog.get(folder_name)[ind])
    print(folder_name)

    #plt.figure(figsize=(15, 15))
    #plt.imshow(v.get_output().get_image())
    #plt.show()
    
    cfg.DATASETS.TRAIN = (folder_name,)
    cfg.DATASETS.TEST = ()
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(folder_name).thing_classes)
    
    torch.cuda.init()
        
    trainer = NonCocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    try:
        trainer.train()
    except:
        continue
    trainer.checkpointer.save(folder_name)

Update dataset
WARNING [06/02 05:33:54 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/02 05:33:54 d2.data.datasets.coco]: Loaded 160 images in COCO format from /host/mic21-framework/server/uploads/figure_skating_gt.json
figure_skating
[06/02 05:33:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=

WARNING [06/02 05:33:55 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/02 05:33:55 d2.data.datasets.coco]: Loaded 160 images in COCO format from /host/mic21-framework/server/uploads/figure_skating_gt.json
[06/02 05:33:55 d2.data.build]: Removed 0 images with no usable annotations. 160 images left.
[06/02 05:33:55 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[06/02 05:33:55 d2.data.build]: Using training sampler TrainingSampler
[06/02 05:33:55 d2.data.common]: Serializing 160 elements to byte tensors and concatenating them all ...
[06/02 05:33:55 d2.data.common]: Serialized dataset takes 1.37 MiB


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

[06/02 05:33:56 d2.engine.train_loop]: Starting training from iteration 0
[06/02 05:34:26 d2.utils.events]:  eta: 0:32:15  iter: 19  total_loss: 2.049  loss_cls: 0.5436  loss_box_reg: 0.9087  loss_mask: 0.5551  loss_rpn_cls: 0.0347  loss_rpn_loc: 0.01717  time: 1.5146  data_time: 0.9344  lr: 0.001  max_mem: 0M
[06/02 05:34:53 d2.utils.events]:  eta: 0:30:32  iter: 39  total_loss: 1.446  loss_cls: 0.3153  loss_box_reg: 0.8168  loss_mask: 0.2698  loss_rpn_cls: 0.01028  loss_rpn_loc: 0.01876  time: 1.4177  data_time: 0.7152  lr: 0.001  max_mem: 0M
[06/02 05:35:23 d2.utils.events]:  eta: 0:31:44  iter: 59  total_loss: 1.047  loss_cls: 0.2041  loss_box_reg: 0.5732  loss_mask: 0.2131  loss_rpn_cls: 0.008094  loss_rpn_loc: 0.01817  time: 1.4495  data_time: 0.8743  lr: 0.001  max_mem: 0M
[06/02 05:35:49 d2.utils.events]:  eta: 0:31:17  iter: 79  total_loss: 0.9116  loss_cls: 0.1671  loss_box_reg: 0.4831  loss_mask: 0.1987  loss_rpn_cls: 0.01156  loss_rpn_loc: 0.01968  time: 1.4094  data_time: 

[06/02 05:50:33 d2.utils.events]:  eta: 0:18:04  iter: 679  total_loss: 0.4798  loss_cls: 0.0853  loss_box_reg: 0.2422  loss_mask: 0.1356  loss_rpn_cls: 0.001803  loss_rpn_loc: 0.01253  time: 1.4656  data_time: 1.9910  lr: 0.001  max_mem: 0M
[06/02 05:51:06 d2.utils.events]:  eta: 0:17:40  iter: 699  total_loss: 0.4595  loss_cls: 0.07793  loss_box_reg: 0.2361  loss_mask: 0.133  loss_rpn_cls: 0.002252  loss_rpn_loc: 0.01179  time: 1.4701  data_time: 0.9877  lr: 0.001  max_mem: 0M
[06/02 05:51:33 d2.utils.events]:  eta: 0:17:10  iter: 719  total_loss: 0.4628  loss_cls: 0.09339  loss_box_reg: 0.2357  loss_mask: 0.1239  loss_rpn_cls: 0.001628  loss_rpn_loc: 0.01174  time: 1.4671  data_time: 0.7309  lr: 0.001  max_mem: 0M
[06/02 05:52:01 d2.utils.events]:  eta: 0:16:41  iter: 739  total_loss: 0.4572  loss_cls: 0.07176  loss_box_reg: 0.2234  loss_mask: 0.1285  loss_rpn_cls: 0.002506  loss_rpn_loc: 0.01127  time: 1.4655  data_time: 0.7579  lr: 0.001  max_mem: 0M
[06/02 05:52:30 d2.utils.event

[06/02 06:10:28 d2.utils.events]:  eta: 0:03:31  iter: 1339  total_loss: 0.37  loss_cls: 0.05556  loss_box_reg: 0.1934  loss_mask: 0.119  loss_rpn_cls: 0.001403  loss_rpn_loc: 0.009069  time: 1.6354  data_time: 1.2361  lr: 5e-05  max_mem: 0M
[06/02 06:11:01 d2.utils.events]:  eta: 0:03:05  iter: 1359  total_loss: 0.3708  loss_cls: 0.05119  loss_box_reg: 0.1828  loss_mask: 0.1035  loss_rpn_cls: 0.001136  loss_rpn_loc: 0.01077  time: 1.6356  data_time: 1.0161  lr: 5e-05  max_mem: 0M
[06/02 06:11:31 d2.utils.events]:  eta: 0:02:39  iter: 1379  total_loss: 0.3904  loss_cls: 0.05467  loss_box_reg: 0.1728  loss_mask: 0.1236  loss_rpn_cls: 0.001969  loss_rpn_loc: 0.009232  time: 1.6332  data_time: 0.8341  lr: 5e-05  max_mem: 0M
[06/02 06:11:56 d2.utils.events]:  eta: 0:02:12  iter: 1399  total_loss: 0.4102  loss_cls: 0.07086  loss_box_reg: 0.1802  loss_mask: 0.1245  loss_rpn_cls: 0.001768  loss_rpn_loc: 0.01087  time: 1.6279  data_time: 0.6120  lr: 5e-05  max_mem: 0M
[06/02 06:12:26 d2.utils.